In [ ]:
!pip install PyGeodesy



In [ ]:
import gpxpy
import pandas as pd

# Load GPX file
file_path = "YPTN_YPKU_41.gpx"
with open(file_path, 'r') as gpx_file:
    gpx = gpxpy.parse(gpx_file)

# Initialize lists to store GPS data
data = {
    'latitude': [],
    'longitude': [],
    'elevation': [],
    'time': []
}

# Extract data from the GPX file
for track in gpx.tracks:
    for segment in track.segments:
        for point in segment.points:
            data['latitude'].append(point.latitude)
            data['longitude'].append(point.longitude)
            data['elevation'].append(point.elevation)
            data['time'].append(point.time)

# Convert the data into a pandas DataFrame
df = pd.DataFrame(data)

# Display the DataFrame
df.head()  # Show first few rows of the DataFrame


import folium

# Create a map centered around the first point
m = folium.Map(location=[df['latitude'].iloc[0], df['longitude'].iloc[0]], zoom_start=13)

# Add a polyline for the track
folium.PolyLine(locations=list(zip(df['latitude'], df['longitude'])), color='blue').add_to(m)

# Add markers for each point (optional)
for idx, row in df.iterrows():
    folium.Marker(location=[row['latitude'], row['longitude']], popup=str(row['time'])).add_to(m)

# Display the map
m.save("track_map.html")  # Save the map to an HTML file
m  # Display the map in a Jupyter Notebook (if applicable)

In [ ]:
from fastkml import kml
import folium

# Load the KML file
kml_file = 'Leg_7.kml'
with open(kml_file, 'rt', encoding='utf-8') as f:
    doc = f.read()

# Parse the KML
k = kml.KML()
k.from_string(doc.encode('utf-8'))

# Create a Folium map
m = folium.Map(location=[-14.512871, 132.368211], zoom_start=15)

# Extract coordinates from KML
for feature in k.features():
    for placemark in feature.features():
        if hasattr(placemark, 'geometry'):
            coords = placemark.geometry.coords
            # Create a list of coordinates for the PolyLine
            points = [(lat, lon) for lon, lat, alt in coords]  # Only take lat and lon
            # Add the PolyLine to the map
            folium.PolyLine(points, color='blue', weight=2.5, opacity=1).add_to(m)

            # Optionally, print or store altitude data
            # altitudes = [alt for lon, lat, alt in coords]
            # print("Altitudes:", altitudes)  # Print altitudes for reference

# Save the map to an HTML file
m.save('map_with_altitude.html')
m

In [ ]:
from fastkml import kml
import folium
import pandas as pd

# Load the KML file
kml_file = 'Leg_7.kml'
with open(kml_file, 'rt', encoding='utf-8') as f:
    doc = f.read()

# Parse the KML
k = kml.KML()
k.from_string(doc.encode('utf-8'))

# Initialize lists to store GPS data
data = {
    'latitude': [],
    'longitude': [],
    'elevation': [],
    'time': []  # Assuming time exists, though it might not in some cases
}

# Recursive function to handle nested KML features and extract data
def extract_features(kml_object):
    for feature in kml_object.features():
        if hasattr(feature, 'geometry') and feature.geometry is not None:
            coords = feature.geometry.coords
            for lon, lat, alt in coords:
                data['latitude'].append(lat)
                data['longitude'].append(lon)
                data['elevation'].append(alt)
                data['time'].append(None)  # Placeholder, you can modify if time data exists
                
        # If there are sub-features, recursively extract them
        if hasattr(feature, 'features'):
            extract_features(feature)

# Start extraction from the top level of the KML object
extract_features(k)

# Convert the data dictionary to a pandas DataFrame
df = pd.DataFrame(data)

# Display the DataFrame
print(df)

# Create a Folium map centered on a default location
m = folium.Map(location=[-14.512871, 132.368211], zoom_start=15)

# Add the GPS track to the map
for i, row in df.iterrows():
    points = [(row['latitude'], row['longitude'])]
    folium.PolyLine(points, color='blue', weight=2.5, opacity=1).add_to(m)

# Save the map to an HTML file
m.save('map_with_altitude.html')

# Display the map (if running in a Jupyter notebook)
m  # Uncomment this line if running in Jupyter


In [ ]:
import gpxpy
import geopandas as gpd
import pandas as pd
from geopy.distance import geodesic
from math import atan2, degrees

# Conversion factors
METERS_TO_FEET = 3.28084
KMH_TO_KNOTS = 0.539957

# Load the GPX file and parse it using gpxpy
gpx_file_path = 'YPTN_YPKU_41.gpx'
with open(gpx_file_path, 'r') as gpx_file:
    gpx = gpxpy.parse(gpx_file)

# Initialize lists to store extracted data
data = {
    'latitude': [],
    'longitude': [],
    'elevation': [],
    'time': [],
    'geometry': []
}

# Extract the track points from the GPX file
for track in gpx.tracks:
    for segment in track.segments:
        for point in segment.points:
            data['latitude'].append(point.latitude)
            data['longitude'].append(point.longitude)
            data['elevation'].append(point.elevation)
            data['time'].append(point.time)
            data['geometry'].append((point.longitude, point.latitude))  # Store geometry as (lon, lat)

# Convert to a pandas DataFrame
df = pd.DataFrame(data)

# Create a GeoDataFrame
gdf = gpd.GeoDataFrame(
    df, geometry=gpd.points_from_xy(df['longitude'], df['latitude'])
)

# Set the CRS to WGS84 (EPSG:4326)
gdf.set_crs(epsg=4326, inplace=True)

# Initialize lists for distance, time difference, speed, elevation change, angle, and converted units
distances = []
time_diffs = []
speeds = []
elevation_changes = []
angles = []

# Compute distance, time difference, speed, elevation change, and angle of elevation between consecutive points
for i in range(1, len(gdf)):
    # Get the current and previous points
    point1 = (gdf.iloc[i-1]['latitude'], gdf.iloc[i-1]['longitude'])
    point2 = (gdf.iloc[i]['latitude'], gdf.iloc[i]['longitude'])

    # Calculate distance using geodesic method (in meters)
    dist = geodesic(point1, point2).meters
    distances.append(dist)

    # Calculate time difference in seconds
    time_diff = (gdf.iloc[i]['time'] - gdf.iloc[i-1]['time']).total_seconds()
    time_diffs.append(time_diff)

    # Calculate speed (m/s) as distance divided by time, and convert to km/h and knots
    if time_diff > 0:
        speed_kmh = (dist / time_diff) * 3.6  # Convert m/s to km/h
        speed_knots = speed_kmh * KMH_TO_KNOTS  # Convert km/h to knots
    else:
        speed_knots = 0
    speeds.append(speed_knots)

    # Calculate elevation change (in meters)
    elevation_change = gdf.iloc[i]['elevation'] - gdf.iloc[i-1]['elevation']
    elevation_changes.append(elevation_change)

    # Calculate angle of elevation (in degrees)
    if dist > 0:  # Avoid division by zero
        angle = degrees(atan2(elevation_change, dist))  # Use atan2 for angle
    else:
        angle = 0
    angles.append(angle)

# Add the calculated values to the GeoDataFrame
gdf['distance_m'] = [0] + distances
gdf['time_diff_sec'] = [0] + time_diffs
gdf['speed_knots'] = [0] + speeds
gdf['elevation_change_m'] = [0] + elevation_changes
gdf['angle_deg'] = [0] + angles

# Convert elevation from meters to feet
gdf['elevation_ft'] = gdf['elevation'] * METERS_TO_FEET
gdf['elevation_change_ft'] = gdf['elevation_change_m'] #iPad gpx file is in knots

# Display the resulting GeoDataFrame
print(gdf)

# Optionally: Save the updated GeoDataFrame to a file or visualize using Folium
import folium

# Create a Folium map centered on the first point
m = folium.Map(location=[gdf.iloc[0]['latitude'], gdf.iloc[0]['longitude']], zoom_start=10)

# Add the GPS track to the map
for i, row in gdf.iterrows():
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=(
            f"Id: {i} \n"
            f"Elevation: {row['elevation_ft']} ft\n"
            f"Speed: {row['speed_knots']} knots\n"
            f"Elevation change: {row['elevation_change_ft']} ft\n"
            f"Angle: {row['angle_deg']}°\n"
            f"Time: {row['time']}\n"
            f"Time_int: {row['time_diff_sec']}"
        )
    ).add_to(m)

# Save the map to an HTML file
m.save('map_with_gpx_data.html')

# Display the map (if running in a Jupyter notebook)
m  # Uncomment if running in Jupyter


In [ ]:
import gpxpy
import gpxpy.gpx

# Parsing an existing file:
# -------------------------

gpx_file = open('YPTN_YPKU_41.gpx', 'r')

gpx = gpxpy.parse(gpx_file)

for waypoint in gpx.waypoints:
    print(f'waypoint {waypoint.name} -> ({waypoint.latitude},{waypoint.longitude})')
    print(f'waypoint {waypoint.name} -> ({waypoint})')
for route in gpx.routes:
    print('Route:')
    for point in route.points:
        print(f'Point at ({point.latitude},{point.longitude}) -> {point.elevation}')
        
for track in gpx.tracks:
    for segment in track.segments:
        for point in segment.points:
            print(f'Point at ({point.latitude},{point.longitude}) -> {point.elevation}')
            for ext in point.extensions:
                for extchild in list(ext):
                    print('{0} -> {1}'.format(extchild.tag, extchild.text))


print('GPX:', gpx.to_xml())

In [ ]:
import gpxpy
import geopandas as gpd
import pandas as pd
from geopy.distance import geodesic
from math import atan2, degrees
import xml.etree.ElementTree as ET
import uuid

# Conversion factors
METERS_TO_FEET = 3.28084
KMH_TO_KNOTS = 0.539956803455724
MPS_TO_KMH = 3.6
MPS_TO_KNOTS = 1.944

# Gate data to load into the DataFrame
gate_points = [
    {'latitude': 34.0522, 'longitude': -118.2437, 'elevation': 100},
    {'latitude': 40.7128, 'longitude': -74.0060, 'elevation': 100}
]
gate_time = 1000

# Parse the GPX file
# Load the GPX file and parse it using gpxpy
gpx_file_path = 'YPTN_YPKU_41.gpx'
with open(gpx_file_path, 'r') as gpx_file:
    gpx = gpxpy.parse(gpx_file)
    
tree = ET.parse(gpx_file_path)
root = tree.getroot()

# Define the namespace for GPX and AvPlan extensions
namespace = {'gpx': 'http://www.topografix.com/GPX/1/1', 'avplan': 'avplan'}

# Initialize a dictionary to store leg data
leg_data = {
    'leg_guid': [],
    'leg_name': [],
    'leg_number': [],
    'src': [],
    'desc': [],
    'departure_time': [],
    'holding_time': [],
    'pob': [],
    'taxi_time': [],
    'block_off': [],
    'wheels_on': [],
    'wheels_off': [],
    'taxi_fuel': [],
    'approach_fuel': [],
    'load_ids': [],
    'load_values': [],
    'fuel_load_ids': [],
    'fuel_load_values': []
}

# Find the <rte> element
for rte in root.findall('gpx:rte', namespace):
    # Generate a GUID for each leg
    leg_guid = str(uuid.uuid4())
    
    # Extract the leg name and number
    leg_name = rte.find('gpx:name', namespace).text if rte.find('gpx:name', namespace) is not None else None
    leg_number = rte.find('gpx:number', namespace).text if rte.find('gpx:number', namespace) is not None else None

    # Extract the <AvPlanLegDetails> extension
    leg_details = rte.find('gpx:extensions/avplan:AvPlanLegDetails', namespace)
    if leg_details is not None:
        src = leg_details.find('avplan:src', namespace).text if leg_details.find('avplan:src', namespace) is not None else None
        desc = leg_details.find('avplan:desc', namespace).text if leg_details.find('avplan:desc', namespace) is not None else None
        departure_time = leg_details.find('avplan:AvPlanDepartureTime', namespace).text if leg_details.find('avplan:AvPlanDepartureTime', namespace) is not None else None
        holding_time = leg_details.find('avplan:AvPlanHoldingTime', namespace).text if leg_details.find('avplan:AvPlanHoldingTime', namespace) is not None else None
        pob = leg_details.find('avplan:AvPlanPOB', namespace).text if leg_details.find('avplan:AvPlanPOB', namespace) is not None else None
        taxi_time = leg_details.find('avplan:AvPlanTaxiTime', namespace).text if leg_details.find('avplan:AvPlanTaxiTime', namespace) is not None else None
        block_off = leg_details.find('avplan:AvPlanBlockOff', namespace).text if leg_details.find('avplan:AvPlanBlockOff', namespace) is not None else None
        wheels_on = leg_details.find('avplan:AvPlanWheelsOn', namespace).text if leg_details.find('avplan:AvPlanWheelsOn', namespace) is not None else None
        wheels_off = leg_details.find('avplan:AvPlanWheelsOff', namespace).text if leg_details.find('avplan:AvPlanWheelsOff', namespace) is not None else None
        taxi_fuel = leg_details.find('avplan:AvPlanTaxiFuel', namespace).text if leg_details.find('avplan:AvPlanTaxiFuel', namespace) is not None else None
        approach_fuel = leg_details.find('avplan:AvPlanApproachFuel', namespace).text if leg_details.find('avplan:AvPlanApproachFuel', namespace) is not None else None

        # Extract loading data
        load_ids = []
        load_values = []
        for load in leg_details.findall('avplan:AvPlanLoading/avplan:AvPlanLoad', namespace):
            load_id = load.find('avplan:AvPlanLoadID', namespace).text if load.find('avplan:AvPlanLoadID', namespace) is not None else None
            load_value = load.find('avplan:AvPlanLoadValue', namespace).text if load.find('avplan:AvPlanLoadValue', namespace) is not None else None
            load_ids.append(load_id)
            load_values.append(load_value)

        # Extract fuel loading data
        fuel_load_ids = []
        fuel_load_values = []
        for fuel_load in leg_details.findall('avplan:AvPlanFuelLoading/avplan:AvPlanFuelLoad', namespace):
            fuel_load_id = fuel_load.find('avplan:AvPlanFuelLoadID', namespace).text if fuel_load.find('avplan:AvPlanFuelLoadID', namespace) is not None else None
            fuel_load_value = fuel_load.find('avplan:AvPlanFuelLoadValue', namespace).text if fuel_load.find('avplan:AvPlanFuelLoadValue', namespace) is not None else None
            fuel_load_ids.append(fuel_load_id)
            fuel_load_values.append(fuel_load_value)

        # Append the extracted data to the dictionary
        leg_data['leg_guid'].append(leg_guid)
        leg_data['leg_name'].append(leg_name)
        leg_data['leg_number'].append(leg_number)
        leg_data['src'].append(src)
        leg_data['desc'].append(desc)
        leg_data['departure_time'].append(departure_time)
        leg_data['holding_time'].append(holding_time)
        leg_data['pob'].append(pob)
        leg_data['taxi_time'].append(taxi_time)
        leg_data['block_off'].append(block_off)
        leg_data['wheels_on'].append(wheels_on)
        leg_data['wheels_off'].append(wheels_off)
        leg_data['taxi_fuel'].append(taxi_fuel)
        leg_data['approach_fuel'].append(approach_fuel)
        leg_data['load_ids'].append(load_ids)
        leg_data['load_values'].append(load_values)
        leg_data['fuel_load_ids'].append(fuel_load_ids)
        leg_data['fuel_load_values'].append(fuel_load_values)

# Convert the dictionary into a pandas DataFrame
leg_df = pd.DataFrame(leg_data)

# Display the DataFrame
print(leg_df)

# Initialize a dictionary to store route point data
route_data = {
    'leg_guid': [],
    'latitude': [],
    'longitude': [],
    'name': [],
    'type': [],
    'altitude': [],
    'description': [],
    'magvar': [],
    'delay': [],
    'alternate': [],
    'lsalt': [],
    'segment_rules': [],
    'track': [],
    'distance': [],
    'distance_remaining': [],
    'eta': [],
    'ata': [],
    'actual_fob': []
}

# Loop through all the <rtept> elements in the GPX file
for rtept in root.findall('gpx:rte/gpx:rtept', namespace):
    lat = rtept.get('lat')
    lon = rtept.get('lon')
    name = rtept.find('gpx:name', namespace).text if rtept.find('gpx:name', namespace) is not None else None
    wpt_type = rtept.find('gpx:type', namespace).text if rtept.find('gpx:type', namespace) is not None else None

    # Find the <AvPlanWaypointDetails> extension within <extensions>
    waypoint_details = rtept.find('gpx:extensions/avplan:AvPlanWaypointDetails', namespace)
    if waypoint_details is not None:
        altitude = waypoint_details.find('avplan:AvPlanAltitude', namespace).text if waypoint_details.find('avplan:AvPlanAltitude', namespace) is not None else None
        description = waypoint_details.find('avplan:desc', namespace).text if waypoint_details.find('avplan:desc', namespace) is not None else None
        magvar = waypoint_details.find('avplan:magvar', namespace).text if waypoint_details.find('avplan:magvar', namespace) is not None else None
        delay = waypoint_details.find('avplan:AvPlanDelay', namespace).text if waypoint_details.find('avplan:AvPlanDelay', namespace) is not None else None
        alternate = waypoint_details.find('avplan:AvPlanAlternate', namespace).text if waypoint_details.find('avplan:AvPlanAlternate', namespace) is not None else None
        lsalt = waypoint_details.find('avplan:AvPlanLSALT', namespace).text if waypoint_details.find('avplan:AvPlanLSALT', namespace) is not None else None
        segment_rules = waypoint_details.find('avplan:AvPlanSegRules', namespace).text if waypoint_details.find('avplan:AvPlanSegRules', namespace) is not None else None
        track = waypoint_details.find('avplan:AvPlanTrack', namespace).text if waypoint_details.find('avplan:AvPlanTrack', namespace) is not None else None
        distance = waypoint_details.find('avplan:AvPlanDistance', namespace).text if waypoint_details.find('avplan:AvPlanDistance', namespace) is not None else None
        distance_remaining = waypoint_details.find('avplan:AvPlanDistanceRemain', namespace).text if waypoint_details.find('avplan:AvPlanDistanceRemain', namespace) is not None else None
        eta = waypoint_details.find('avplan:AvPlanETA', namespace).text if waypoint_details.find('avplan:AvPlanETA', namespace) is not None else None
        ata = waypoint_details.find('avplan:AvPlanATA', namespace).text if waypoint_details.find('avplan:AvPlanATA', namespace) is not None else None
        actual_fob = waypoint_details.find('avplan:AvPlanActualFOB', namespace).text if waypoint_details.find('avplan:AvPlanActualFOB', namespace) is not None else None

        # Append the extracted data to the dictionary
        route_data['leg_guid'].append(leg_guid)
        route_data['latitude'].append(float(lat))
        route_data['longitude'].append(float(lon))
        route_data['name'].append(name)
        route_data['type'].append(wpt_type)
        route_data['altitude'].append(float(altitude) if altitude else None)
        route_data['description'].append(description)
        route_data['magvar'].append(float(magvar) if magvar else None)
        route_data['delay'].append(float(delay) if delay else None)
        route_data['alternate'].append(float(alternate) if alternate else None)
        route_data['lsalt'].append(float(lsalt) if lsalt else None)
        route_data['segment_rules'].append(segment_rules)
        route_data['track'].append(float(track) if track else None)
        route_data['distance'].append(float(distance) if distance else None)
        route_data['distance_remaining'].append(float(distance_remaining) if distance_remaining else None)
        route_data['eta'].append(eta)
        route_data['ata'].append(ata)
        route_data['actual_fob'].append(float(actual_fob) if actual_fob else None)

# Convert the dictionary into a pandas DataFrame
route_df = pd.DataFrame(route_data)

# Display the DataFrame
print(route_df)


# Initialize lists to store extracted data
data = {
    'leg_guid': [],
    'latitude': [],
    'longitude': [],
    'elevation_ft': [],
    'time': [],
    'speed_avplan': [],
    'heading': [],
    'hdop': [],
    'vdop': [],
    'geometry': []
}

# Extract the track points from the GPX file
for track in gpx.tracks:
    for segment in track.segments:
        for point in segment.points:
            data['leg_guid'].append(leg_guid)
            data['latitude'].append(point.latitude)
            data['longitude'].append(point.longitude)
            data['elevation_ft'].append(point.elevation)  # Assume elevation is already in feet
            data['time'].append(point.time)

            # Extract HDOP, VDOP, and heading from the extensions if available
            hdop = None
            vdop = None
            heading = None
            speed_knots = None

            if point.extensions:
                for ext in point.extensions:
                    tree = ET.ElementTree(ET.fromstring(ET.tostring(ext)))
                    root = tree.getroot()

                    # Extract speed (knots), heading, hdop, and vdop from extensions
                    speed_elem = root.find('.//{avplan}speed')
                    heading_elem = root.find('.//{avplan}heading')
                    hdop_elem = root.find('.//hdop')
                    vdop_elem = root.find('.//vdop')

                    if speed_elem is not None:
                        speed_avplan = float(speed_elem.text)
                    if heading_elem is not None:
                        heading = float(heading_elem.text)
                    if hdop_elem is not None:
                        hdop = float(hdop_elem.text)
                    if vdop_elem is not None:
                        vdop = float(vdop_elem.text)

            data['speed_avplan'].append(speed_avplan)
            data['heading'].append(heading)
            data['hdop'].append(hdop)
            data['vdop'].append(vdop)
            data['geometry'].append((point.longitude, point.latitude))  # Store geometry as (lon, lat)

# Convert to a pandas DataFrame
df = pd.DataFrame(data)

# Create a GeoDataFrame
gdf = gpd.GeoDataFrame(
    df, geometry=gpd.points_from_xy(df['longitude'], df['latitude'])
)

# Set the CRS to WGS84 (EPSG:4326)
gdf.set_crs(epsg=4326, inplace=True)

# Initialize lists for distance, time difference, elevation change, and angle of elevation
distances = []
speed_calc = []
time_diffs = []
elevation_changes = []
angles = []

# Compute distance, time difference, elevation change, and angle of elevation between consecutive points
for i in range(1, len(gdf)):
    # Get the current and previous points
    point1 = (gdf.iloc[i-1]['latitude'], gdf.iloc[i-1]['longitude'])
    point2 = (gdf.iloc[i]['latitude'], gdf.iloc[i]['longitude'])

    # Calculate distance using geodesic method (in meters)
    dist = geodesic(point1, point2).meters
    distances.append(dist)

    # Calculate time difference in seconds
    time_diff = (gdf.iloc[i]['time'] - gdf.iloc[i-1]['time']).total_seconds()
    time_diffs.append(time_diff)
    
    # Calculate speed (m/s) as distance divided by time, and convert to km/h and knots
    if time_diff > 0:
        speed_mps = (dist / time_diff)
        speed_kmh =  speed_mps * MPS_TO_KMH  # Convert m/s to km/h
        speed_knots = speed_kmh * KMH_TO_KNOTS  # Convert km/h to knots
    else:
        speed_knots = 0
    speed_calc.append(speed_mps)

    # Calculate elevation change (in feet)
    elevation_change = gdf.iloc[i]['elevation_ft'] - gdf.iloc[i-1]['elevation_ft']
    elevation_changes.append(elevation_change)

    # Calculate angle of elevation (in degrees)
    if dist > 0:  # Avoid division by zero
        angle = degrees(atan2(elevation_change, dist))  # Use atan2 for angle
    else:
        angle = 0
    angles.append(angle)

# Add the calculated values to the GeoDataFrame
gdf['distance_m'] = [0] + distances
gdf['speed_calc'] = [0] + speed_calc
gdf['time_diff_sec'] = [0] + time_diffs
gdf['elevation_change_ft'] = [0] + elevation_changes
gdf['angle_deg'] = [0] + angles

# Display the resulting GeoDataFrame
print(gdf)



In [18]:
import gpxpy
import geopandas as gpd
import pandas as pd
from pygeodesy.ellipsoidalVincenty import LatLon
from pygeodesy import sphericalTrigonometry as st
from geopy.distance import geodesic
from math import atan2, degrees
import folium
import xml.etree.ElementTree as ET
import re

# Function to find the two closest points on the track
def find_closest_points(gate_point, gdf):
    gate_lat, gate_lon = gate_point['latitude'], gate_point['longitude']
    
    # Calculate distances from the gate_point to all points in the GeoDataFrame
    distances = gdf.apply(lambda row: geodesic((gate_lat, gate_lon), (row['latitude'], row['longitude'])).meters, axis=1)

    # Find the index of the closest point
    closest_index = distances.idxmin()


    # Find the second & third closest point (before and after the closest)
    if closest_index == 0:
        closest_index = 1
        second_closest_index = 0
        third_closest_index = 2
    elif closest_index == len(gdf) - 1:
        closest_index = len(gdf) - 2
        second_closest_index = len(gdf) - 3
        third_closest_index = -1
    else:
        second_closest_index = closest_index - 1
        third_closest_index = closest_index + 1
    
    return closest_index, second_closest_index, third_closest_index



def interpolate_between_points(gate_point, gdf, idx1, idx2, idx3):
    """
    Interpolate the closest point on the geodesic line (great circle) between two track points to the gate point.
    Uses PyGeodesy to calculate the closest point along the geodesic line.
    """
    # Extract the two closest points from the GeoDataFrame
    point1 = gdf.iloc[idx1]
    point2 = gdf.iloc[idx2]
    point3 = gdf.iloc[idx3]
    
    # Use PyGeodesy to find the nearest point on the geodesic between point1 and point2
    #nearest = gate_geo.nearestOn3(point1_geo, point2_geo)
    
    # Create PyGeodesy LatLon objects for the track points and gate point
    point1_geo = st.LatLon(point1['latitude'], point1['longitude'])
    point2_geo = st.LatLon(point2['latitude'], point2['longitude'])
    point3_geo = st.LatLon(point3['latitude'], point3['longitude'])
    gate_geo = st.LatLon(gate_point['latitude'], gate_point['longitude'])

    # Use PyGeodesy to find the nearest point on the geodesic between point1 and point2
    #nearest = gate_geo.nearestOn(point1_geo, point2_geo, point3_geo)
    #print(nearest)
    nearest = [st.nearestOn3(gate_geo, [point1_geo,point2_geo,point3_geo])] 
    nearest_geo = nearest[0].closest
    
    # Interpolate time between the two points
    time_diff = (point3['time'] - point2['time']).total_seconds()
    dist_diff = point3_geo.distanceTo(point2_geo)
    dist_to = point2_geo.distanceTo(nearest_geo)
    fraction_of_distance = dist_to/dist_diff
    interpolated_time = point2['time'] + pd.Timedelta(seconds=time_diff * fraction_of_distance)

    # Compute distance from the gate point to the nearest point on the geodesic
    distance_from_gate = gate_geo.distanceTo(nearest_geo)

    # Return the results in a dictionary
    return {
        'gate_lat': gate_point['latitude'],
        'gate_lon': gate_point['longitude'],
        'interpolated_lat': nearest_geo.lat,
        'interpolated_lon': nearest_geo.lon,
        'interpolated_time': interpolated_time,
        'distance_from_gate_m': distance_from_gate
    }


# Function to process each gate point and interpolate
def compute_track_points_for_gate(gate_points, gdf, map_obj):
    interpolated_results = []

    for gate_point in gate_points:
        idx1, idx2, idx3 = find_closest_points(gate_point, gdf)
        result = interpolate_between_points(gate_point, gdf, idx1, idx2, idx3)

        # Add interpolated point to the map (red marker)
        folium.Marker(
            location=[result['interpolated_lat'], result['interpolated_lon']],
            popup=(
                f"Interpolated Point<br>Coordinates: ({result['interpolated_lat']}, {result['interpolated_lon']})<br>"
                f"Time: {result['interpolated_time']}<br>"
                f"Distance from gate: {result['distance_from_gate_m']} meters"
            ),
            icon=folium.Icon(color='red')
        ).add_to(map_obj)

        interpolated_results.append(result)

    return pd.DataFrame(interpolated_results)

# Function to check if a name starts with "in" or "out" followed by a number
def is_in_or_out_with_number(name):
    # Use regex to match "in" or "out" followed by a number
    return bool(re.match(r'^(in|out)\d+', name.lower()))

# Extract the gate points from the route data where name matches the pattern
gate_points = []
for index, row in route_df.iterrows():
    if is_in_or_out_with_number(row['name']):
        # Create a dictionary with the latitude, longitude, and elevation
        gate_point = {
            'latitude': row['latitude'],
            'longitude': row['longitude'],
            'elevation': row['altitude']  # Assuming 'altitude' is the correct field for elevation
        }
        # Add the gate point to the list
        gate_points.append(gate_point)

# Assuming gdf is already created with track points
# Optionally: Save the updated GeoDataFrame to a file or visualize using Folium


# Create a Folium map centered on the first point
m = folium.Map(location=[gdf.iloc[0]['latitude'], gdf.iloc[0]['longitude']], zoom_start=10)

# Function to check if a name starts with "in" or "out" followed by a number
def is_in_or_out_with_number(name):
    # Use regex to match "in" or "out" followed by a number
    return bool(re.match(r'^(in|out)\d+', name.lower()))

# Plot each route point
for index, row in route_df.iterrows():
    # Determine the icon color: red for points with names starting with "in" or "out" followed by a number, otherwise orange
    icon_color = 'green' if is_in_or_out_with_number(row['name']) else 'orange'
    
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=(f"Name: {row['name']}<br>Type: {row['type']}<br>Altitude: {row['altitude']} ft<br>Description: {row['description']}"),
        icon=folium.Icon(color=icon_color, icon='info-sign')
    ).add_to(m)
    
# Add the GPS track to the map
for i, row in gdf.iterrows():
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=(
            f"Id: {i} <pr>"
            f"Elevation: {row['elevation_ft']} ft\n"
            f"Speed Av: {row['speed_avplan'] * MPS_TO_KNOTS} knots\n"
            f"Speed Cal: {row['speed_calc'] * MPS_TO_KNOTS} knots\n"
            f"Heading: {row['heading']}°\n"
            f"Distance: {row['distance_m']} m\n"
            f"Interval: {row['time_diff_sec']}\n"
            f"Elevation change: {row['elevation_change_ft']} ft\n"
            f"Angle: {row['angle_deg']}°\n"
            f"Time: {row['time']}")
    ).add_to(m)

# Save the map to an HTML file
m.save('map_with_gpx_data.html')

# Display the map (if running in a Jupyter notebook)
m  # Uncomment if running in Jupyter


# Call the function to compute track points and plot them on the map
interpolated_gate_points = compute_track_points_for_gate(gate_points, gdf, m)

# Save the map to an HTML file
m.save('map_with_gates_and_interpolated_points.html')

# Display the resulting DataFrame with interpolated data
print(interpolated_gate_points['distance_from_gate_m'])
print(interpolated_gate_points['interpolated_time'].diff())
m

NameError: name 'route_df' is not defined

In [11]:
import tkinter as tk
from tkinter import filedialog

def select_file():
    # Initialize the Tkinter root window
    root = tk.Tk()
    root.withdraw()  # Hide the root window
    
    # Open the file selection dialog
    file_path = filedialog.askopenfilename(
        title="Select a file",
        filetypes=[("All Files", "*.*"), ("Text Files", "*.txt"), ("Python Files", "*.py")]
    )
    
    # Print the selected file path
    if file_path:
        print(f"Selected file: {file_path}")
    else:
        print("No file selected")

# Call the file selection dialog
select_file()


No file selected


In [53]:
import sys
import struct
import math
import time
import datetime
import argparse
import pandas as pd

# Constants and Definitions
HEADER_LEN = 8
SIGNATURE_LEN = 2
PRIMARY_FLIGHT_DATA_LEN = 24
RDAC1_DATA_LEN = 56
RDAC1_DATA_LEN_REV_2 = 63
RDAC2_DATA_LEN = RDAC1_DATA_LEN
RDAC2_DATA_LEN_REV_2 = RDAC1_DATA_LEN_REV_2
ATTITUDE_DATA_LEN = 14
GPS_DATA_LEN_REV_0 = 20
GPS_DATA_LEN_REV_1 = 24

MAX_PACKET = 1000  # larger than any legal packet

PACKET_SIGNATURE_1 = 0xaa
PACKET_SIGNATURE_2 = 0x55
SIGNATURE_LEN = 2

TIMESTAMP_LEN = 4

MAX_REVISION = 3  # counting from zero

delimiters = {
    "tab": "\t",
    "comma": ",",
    "space": " ",
    "no_delim": "",
}

fudge_factors = [6, 2, 2, 6]  # length is MAX_REVISION+1 = 4

primary_section = 0
rdac1_section = 1
rdac2_section = 2
attitude_section = 3
gps_section = 4

section_sizes = [
    [24, 24, 24, 24],  # primary
    [56, 56, 63, 63],  # rdac1
    [56, 56, 63, 63],  # rdac2
    [14, 14, 14, 14],  # attitude
    [20, 24, 24, 24],  # GPS
]

rdac1_data_tag = 1
rdac2_data_tag = 2
attitude_data_tag = 3
gps_data_tag = 4

RDAC1_PRESENT = 1 << rdac1_data_tag  # (1 << 1) == 2
RDAC2_PRESENT = 1 << rdac2_data_tag  # (1 << 2) == 4
ATTITUDE_PRESENT = 1 << attitude_data_tag  # (1 << 3) == 8
GPS_PRESENT = 1 << gps_data_tag  # (1 << 4) == 16


class EnigmaLogParser:
    def __init__(self, filename):
        self.logfile = open(filename, 'rb')
        self.bad_packets = 0
        self.reversals = 0

    def read_packet(self):
        packet = bytearray()
        processed = 0
        while True:
            i1 = self.logfile.read(1)
            if not i1:
                print("rp_EOF_i1", end='')    
                return None  # EOF
            if i1[0] == PACKET_SIGNATURE_1:
                i2 = self.logfile.read(1)
                if not i2:
                    print("rp_EOF_i2", end='')      
                    return None  # EOF
                if i2[0] == PACKET_SIGNATURE_2:
                    # Valid starting signature, read rest of packet
                    packet_body = self.read_body()
                    if packet_body is None:
                        return None
                    packet.extend(i1)
                    packet.extend(i2)
                    packet.extend(packet_body)
                    return packet
                else:
                    # Move back one byte
                    self.logfile.seek(-1, 1)
            processed += 1
            if processed % 100 == 0:
                print("rp", end='')
            
        return None  # Should not reach here

    def read_body(self):
        packet_body = bytearray()
        processed = 0
        while True:
            i1 = self.logfile.read(1)
            if not i1:
                break  # EOF
            if i1[0] == PACKET_SIGNATURE_1:
                i2 = self.logfile.read(1)
                if not i2:
                    break  # EOF
                if i2[0] == PACKET_SIGNATURE_2:
                    # Signature of next packet, back up and exit
                    self.logfile.seek(-2, 1)
                    break
                else:
                    # False trigger, save byte and continue
                    packet_body.append(i1[0])
                    self.logfile.seek(-1, 1)
            else:
                packet_body.append(i1[0])
                
            processed += 1
            if processed % 1000000 == 0:
                print(f'b', end='')
                
  
        return packet_body

    def read_and_id_packet(self):
        while True:
            packet = self.read_packet()
            if packet is None:
                print("rp_none", end='')
            
                return None
            packet_len = len(packet)
            if packet_len < HEADER_LEN:
                continue  # Try next packet

            timestamp = (
                packet[7] << 24 |
                packet[6] << 16 |
                packet[5] << 8 |
                packet[4]
            )
            # Extract bytes from index 4 to 7 (inclusive)
            bytes_4 = packet[3:7]  # Slices from index 3 to 6
            # Convert the extracted 4 bytes to an integer (big-endian or little-endian)
            #timestamp = int.from_bytes(bytes_4, byteorder='little') # 'big' or 'little'
            
            payload = packet
            # packet[0] is the total payload length + fudge factor
            # packet[1] is an offset

            # Convert each byte to its hexadecimal representation and join them with a space
            hex_output = ' '.join(f'{byte:02X}' for byte in packet)

            # Print the result
#            print(f'Timestamp: {timestamp} for Packet:{hex_output}')  
            for revision in range(MAX_REVISION + 1):
                sections_seen = 0
                rdac1_len = 0
                rdac2_len = 0
                attitude_len = 0
                gps_len = 0

                if revision >= len(fudge_factors):
                    continue

                if packet[0] < fudge_factors[revision]:
                    continue

                payload_len = packet[2] + SIGNATURE_LEN

                #if (payload_len < PRIMARY_FLIGHT_DATA_LEN or
                #        payload_len + HEADER_LEN - SIGNATURE_LEN != packet_len):
                #    continue

                p_offset = section_sizes[0][revision]+ HEADER_LEN + fudge_factors[revision]
                if p_offset < 0 or p_offset > len(payload):
                    continue  # Invalid offset

                p_idx = p_offset

                # Parse sections
                if p_idx < payload_len and payload[p_idx] == rdac1_data_tag:
                    sections_seen |= RDAC1_PRESENT
                    tag = payload[p_idx]
                    p_idx += 1
                    rdac1_len = payload[p_idx]
                    p_idx += 1
                    p_idx += rdac1_len

                if p_idx < payload_len and payload[p_idx] == rdac2_data_tag:
                    sections_seen |= RDAC2_PRESENT
                    tag = payload[p_idx]
                    p_idx += 1
                    rdac2_len = payload[p_idx]
                    p_idx += 1
                    p_idx += rdac2_len

                if p_idx < payload_len and payload[p_idx] == attitude_data_tag:
                    sections_seen |= ATTITUDE_PRESENT
                    tag = payload[p_idx]
                    p_idx += 1
                    attitude_len = payload[p_idx]
                    p_idx += 1
                    p_idx += attitude_len

                if p_idx < payload_len and payload[p_idx] == gps_data_tag:
                    sections_seen |= GPS_PRESENT
                    tag = payload[p_idx]
                    p_idx += 1
                    gps_len = payload[p_idx]
                    p_idx += 1
                    p_idx += gps_len

                if (p_idx == payload_len and
                    ((sections_seen & RDAC1_PRESENT == 0) or rdac1_len == section_sizes[rdac1_section][revision]) and
                    ((sections_seen & RDAC2_PRESENT == 0) or rdac2_len == section_sizes[rdac2_section][revision]) and
                    ((sections_seen & ATTITUDE_PRESENT == 0) or attitude_len == section_sizes[attitude_section][revision]) and
                    ((sections_seen & GPS_PRESENT == 0) or gps_len == section_sizes[gps_section][revision])
                   ):
                    #print(payload, payload_len, revision, sections_seen)
                    return timestamp, payload[HEADER_LEN:], payload_len, revision, sections_seen

            self.bad_packets += 1
            # Try next packet

    def find_first_last(self):
        oldest_pos = -1
        newest_pos = -1
        sections_seen = 0
        reversals = 0

        self.logfile.seek(0, 0)
        cur_pos = self.logfile.tell()
        oldest = 4294967295 # For 32-bit unsigned long
        newest = 0
        last = 0

        processed = 0

        while True:
            pos = self.logfile.tell()
            result = self.read_and_id_packet()
            #print("fl_it", end='')
            if result is None:
                print("fl_break", end='')
            
                break
            timestamp, payload, payload_len, revision, one_packet_sections_seen = result
            if processed:
                if sections_seen != one_packet_sections_seen:
                    print("\nWarning, records contain inconsistent sections\n")
                
            else:
                pass  # First packet

            sections_seen |= one_packet_sections_seen
            #print(f'Timestamp: {timestamp}')
        
            if timestamp < oldest:
                oldest = timestamp
                oldest_pos = pos

            if timestamp > newest:
                newest = timestamp
                newest_pos = pos

            if timestamp < last:
                reversals += 1

            last = timestamp

            processed += 1
            if processed % 1000 == 0:
                print("f", end='')
            

        return oldest_pos, newest_pos, sections_seen, reversals

    def rdac_headers(self):
        headers = ["RPM", "rfl1", "rfl2", "ch1", "ch2", "flow", "MAP", "fl1", "fl2", "flc",
                   "o-t", "oil-p", "carb", "f-p", "h2o"]
        headers.extend([f"tc{i}" for i in range(1, 13)])
        headers.extend(["rch1", "rch2", "rot", "rop", "ref", "fail"])
        return headers

    def print_header(self, delimiter_type, sections_seen):
        delimiter = delimiters[delimiter_type]
        headers = []
        headers.append("Time")

        headers.extend(["alt", "baro", "ASI", "TAS", "VSI", "g-s", "rotor", "m-v", "b-v", "amps", "AOA", "OAT"])

        if sections_seen & RDAC1_PRESENT:
            headers.extend(self.rdac_headers())
        if sections_seen & RDAC2_PRESENT:
            headers.extend(self.rdac_headers())
        if sections_seen & ATTITUDE_PRESENT:
            headers.extend(["bank", "ptch", "slip", "hdg", "yaw", "G", "turn"])
        if sections_seen & GPS_PRESENT:
            headers.extend(["lat", "long", "hdg", "gs", "g-alt", "status", "sats", "hac", "vac"])

        #print(delimiter.join(headers))
        file.write(delimiter.join(headers) + "\n")  # Write headers to the file
        return headers


    def process_packets(self, delimiter_type, stop_pos, sections_seen):
        delimiter = delimiters[delimiter_type]
        packets = 0
        mgl_records = []
        while self.logfile.tell() < stop_pos:
            result = self.read_and_id_packet()
            if result is None:
                break
            timestamp, payload, payload_len, revision, packet_sections_seen = result
            p = payload
            idx = 0
            # Set the block size (number of bytes per line)
            block_size = 10
            # Convert each byte to its hexadecimal representation and join them with a space
            hex_output = ' '.join(f'{byte:02X}' for byte in p)
            # Split the hex output into blocks of 'block_size' and print each block on a new line
            hex_blocks = hex_output.split()  # Convert hex string to a list of hex values
            #print(f'{len(p)} ', end='')
            #for i in range(0, len(hex_blocks), block_size):
            #    # Print each block (8 hex values per line)
            #    print(' '.join(hex_blocks[i:i + block_size]))
            valid = True
            #print(f'T:{timestamp} - Len:{payload_len} - Rev:{revision}')
            # Time processing
            if revision >= 2:
                base_time = datetime.datetime(2000, 1, 1)
                adj_time = base_time + datetime.timedelta(seconds=timestamp)
                time_str = adj_time.strftime("%Y/%m/%d %H:%M:%S")
                output = [f'"{time_str}"']
            else:
                output = [str(timestamp)]

            # Primary Flight Data
            # Using struct.unpack for binary data parsing
            try:
                # Altitude (S32)
                alt, = struct.unpack_from('<i', p, idx)
                idx += 4
                output.append(str(alt))
                # Barometer (S16)
                baro, = struct.unpack_from('<h', p, idx)
                idx += 2
                output.append(str(baro))
                # Airspeed (S16)
                asi, = struct.unpack_from('<h', p, idx)
                idx += 2
                output.append(str(asi))
                # TAS (S16)
                tas, = struct.unpack_from('<h', p, idx)
                idx += 2
                output.append(str(tas))
                # VSI (S16)
                vsi, = struct.unpack_from('<h', p, idx)
                idx += 2
                output.append(str(vsi))
                # Glide Slope (S16 tenths)
                gs, = struct.unpack_from('<h', p, idx)
                idx += 2
                gs_tenths = gs / 10.0
                output.append(f"{gs_tenths:.1f}")
                # Rotor RPM (U16)
                rotor_raw, = struct.unpack_from('<H', p, idx)
                idx += 2
                rotor = rotor_raw & 0x7fff
                output.append(str(rotor))
                # Main Voltage (U8 tenths)
                mv_raw, = struct.unpack_from('<B', p, idx)
                idx += 1
                mv = mv_raw / 10.0
                output.append(f"{mv:.1f}")
                # Backup Voltage (U8 tenths)
                bv_raw, = struct.unpack_from('<B', p, idx)
                idx += 1
                bv = bv_raw / 10.0
                output.append(f"{bv:.1f}")
                # Current (S16 tenths)
                amps_raw, = struct.unpack_from('<h', p, idx)
                idx += 2
                amps = amps_raw / 10.0
                output.append(f"{amps:.1f}")
                # AOA (S16)
                aoa, = struct.unpack_from('<h', p, idx)
                idx += 2
                output.append(str(aoa))
                # OAT (S16)
                oat, = struct.unpack_from('<h', p, idx)
                idx += 2
                output.append(str(oat))
            except struct.error:
                self.bad_packets += 1
                continue  # Skip to next packet
            idx += fudge_factors[revision] #Seem to need to skip some extra bytes here
            # RDAC Sections
            for rdac_tag in [rdac1_data_tag, rdac2_data_tag]:
                if sections_seen & (1 << rdac_tag):
                    if idx < len(p) and p[idx] == rdac_tag:
                        # Unpack tag and length
                        tag, length = struct.unpack_from('<BB', p, idx)
                        idx += 2  # Move index past tag and length

                        # Direct use of struct.unpack_from for each data field
                        # RPM (16-bit unsigned)
                        rpm = struct.unpack_from('<H', p, idx)[0] if valid else 0
                        idx += 2
                        output.append(str(rpm))

                        # Tank 1 raw (16-bit unsigned)
                        tank1_raw = struct.unpack_from('<H', p, idx)[0] if valid else 0
                        idx += 2
                        output.append(str(tank1_raw))

                        # Tank 2 raw (16-bit unsigned)
                        tank2_raw = struct.unpack_from('<H', p, idx)[0] if valid else 0
                        idx += 2
                        output.append(str(tank2_raw))

                        # Rotax CHT 1 (16-bit unsigned)
                        rotax_cht1 = struct.unpack_from('<H', p, idx)[0] if valid else 0
                        idx += 2
                        output.append(str(rotax_cht1))

                        # Rotax CHT 2 (16-bit unsigned)
                        rotax_cht2 = struct.unpack_from('<H', p, idx)[0] if valid else 0
                        idx += 2
                        output.append(str(rotax_cht2))

                        # Fuel flow (16-bit unsigned, divided by 100)
                        fuel_flow = struct.unpack_from('<H', p, idx)[0] / 100.0 if valid else 0.0
                        idx += 2
                        output.append(f"{fuel_flow:.2f}")

                        # MAP (Manifold Absolute Pressure, 16-bit unsigned)
                        map_val = struct.unpack_from('<H', p, idx)[0] if valid else 0
                        idx += 2
                        output.append(str(map_val))

                        # Fuel level 1 (16-bit unsigned)
                        fuel_level1 = struct.unpack_from('<H', p, idx)[0] if valid else 0
                        idx += 2
                        output.append(str(fuel_level1))

                        # Fuel level 2 (16-bit unsigned)
                        fuel_level2 = struct.unpack_from('<H', p, idx)[0] if valid else 0
                        idx += 2
                        output.append(str(fuel_level2))

                        # Calculated fuel level (16-bit unsigned, divided by 10)
                        calc_fuel_level = struct.unpack_from('<H', p, idx)[0] / 10.0 if valid else 0.0
                        idx += 2
                        output.append(f"{calc_fuel_level:.1f}")

                        # Oil temperature (16-bit unsigned)
                        oil_temp = struct.unpack_from('<H', p, idx)[0] if valid else 0
                        idx += 2
                        output.append(str(oil_temp))

                        # Oil pressure (16-bit unsigned, divided by 10)
                        oil_pressure = struct.unpack_from('<H', p, idx)[0] / 10.0 if valid else 0.0
                        idx += 2
                        output.append(f"{oil_pressure:.1f}")

                        # Carb temperature (16-bit signed)
                        carb_temp = struct.unpack_from('<h', p, idx)[0] if valid else 0
                        idx += 2
                        output.append(str(carb_temp))

                        # Fuel pressure (8-bit unsigned, divided by 10)
                        fuel_pressure = struct.unpack_from('<B', p, idx)[0] / 10.0 if valid else 0.0
                        idx += 1
                        output.append(f"{fuel_pressure:.1f}")

                        # Water temperature (8-bit unsigned)
                        water_temp = struct.unpack_from('<B', p, idx)[0] if valid else 0
                        idx += 1
                        output.append(str(water_temp))

                        # Process 12 thermocouples (each 16-bit unsigned)
                        for i in range(12):
                            thermocouple = struct.unpack_from('<H', p, idx)[0] if valid else 0
                            idx += 2
                            output.append(str(thermocouple))

                        # Additional fields if revision == 2
                        if revision == 2:
                            raw_cht1 = struct.unpack_from('<H', p, idx)[0] if valid else 0
                            idx += 2
                            output.append(str(raw_cht1))

                            raw_cht2 = struct.unpack_from('<H', p, idx)[0] if valid else 0
                            idx += 2
                            output.append(str(raw_cht2))

                            raw_oil_temp = struct.unpack_from('<H', p, idx)[0] if valid else 0
                            idx += 2
                            output.append(str(raw_oil_temp))

                            raw_oil_pressure = struct.unpack_from('<H', p, idx)[0] if valid else 0
                            idx += 2
                            output.append(str(raw_oil_pressure))

                            rdac_temp = struct.unpack_from('<H', p, idx)[0] if valid else 0
                            #Result Doesn't compare to the MGL program e.g 2403 vs 21. Reads the bits correctly.
                            idx += 2
                            output.append(str(rdac_temp))

                            rdac_fail = struct.unpack_from('<B', p, idx)[0] if valid else 0
                            idx += 1
                            output.append(str(rdac_fail))

                        else:
                            # If revision is not 2, treat raw CHT and oil fields as invalid (set valid=False)
                            raw_cht1 = struct.unpack_from('<H', p, idx)[0] if False else 0
                            idx += 2
                            output.append(str(raw_cht1))

                            raw_cht2 = struct.unpack_from('<H', p, idx)[0] if False else 0
                            idx += 2
                            output.append(str(raw_cht2))

                            raw_oil_temp = struct.unpack_from('<H', p, idx)[0] if False else 0
                            idx += 2
                            output.append(str(raw_oil_temp))

                            raw_oil_pressure = struct.unpack_from('<H', p, idx)[0] if False else 0
                            idx += 2
                            output.append(str(raw_oil_pressure))

                            rdac_temp = struct.unpack_from('<H', p, idx)[0] if valid else 0
                            idx += 2
                            output.append(str(rdac_temp))

                            rdac_fail = struct.unpack_from('<B', p, idx)[0] if valid else 0
                            idx += 1
                            output.append(str(rdac_fail))

                            # Move past padding byte if present
                            idx += 1
                    else:
                        output.extend(['3'] * 39)

            # Attitude Section
            if sections_seen & ATTITUDE_PRESENT:
                if idx < len(p) and p[idx] == attitude_data_tag:
                    # Unpack tag and length
                    tag, length = struct.unpack_from('<BB', p, idx)
                    idx += 2  # Move past tag and length

                    # Bank angle (signed 16-bit integer)
                    bank = struct.unpack_from('<h', p, idx)[0] if valid else 0
                    idx += 2
                    output.append(str(bank))

                    # Pitch angle (signed 16-bit integer)
                    pitch = struct.unpack_from('<h', p, idx)[0] if valid else 0
                    idx += 2
                    output.append(str(pitch))

                    # Slip (signed 16-bit integer)
                    slip = struct.unpack_from('<h', p, idx)[0] if valid else 0
                    idx += 2
                    output.append(str(slip))

                    # Magnetic heading (signed 16-bit integer)
                    magnetic_heading = struct.unpack_from('<h', p, idx)[0] if valid else 0
                    idx += 2
                    output.append(str(magnetic_heading))

                    # Yaw/gyro heading (unsigned 16-bit integer)
                    yaw_gyro_heading = struct.unpack_from('<H', p, idx)[0] if valid else 0
                    idx += 2
                    output.append(str(yaw_gyro_heading))

                    # G-force (signed 8-bit integer, interpreted in tenths)
                    g_force = struct.unpack_from('<b', p, idx)[0] / 10.0 if valid else 0.0
                    idx += 1
                    output.append(f"{g_force:.1f}")

                    # Turn rate (Z-format MMSS: unsigned byte of seconds followed by signed word of minutes)
                    turn_rate_seconds = struct.unpack_from('<B', p, idx)[0] if valid else 0  # unsigned 8-bit integer for seconds
                    idx += 1
                    turn_rate_minutes = struct.unpack_from('<h', p, idx)[0] if valid else 0  # signed 16-bit integer for minutes
                    idx += 2

                    # Format and append turn rate in MM:SS format
                    turn_rate = f"{turn_rate_minutes}:{turn_rate_seconds:02d}"
                    output.append(turn_rate)
                    
                else:
                    output.extend(['7'] * 7)

            # GPS Section
            if sections_seen & GPS_PRESENT:
                if idx < len(p) and p[idx] == gps_data_tag:          
                    # Unpack GPS tag and length (both are unsigned 8-bit integers)
                    tag, length = struct.unpack_from('<BB', p, idx)
                    idx += 2  # Move index past tag and length

                    # Interpret latitude as a 32-bit float (4 bytes)
                    latitude, = struct.unpack_from('<f', p, idx)
                    idx += 4
                    output.append(f"{latitude:.5f}")  # Append latitude to output

                    # Interpret longitude as a 32-bit float (4 bytes), negated (multiply by -1)
                    longitude, = struct.unpack_from('<f', p, idx)
                    idx += 4
                    output.append(f"{longitude:.5f}")  # Append negated longitude

                    # Interpret GPS heading as signed 32-bit integer
                    heading, = struct.unpack_from('<i', p, idx)
                    idx += 4
                    output.append(str(heading))  # Append GPS heading to output

                    # Interpret ground speed as signed 32-bit integer
                    ground_speed, = struct.unpack_from('<i', p, idx)
                    idx += 4
                    output.append(str(ground_speed))  # Append ground speed to output

                    # Interpret altitude as signed 32-bit integer
                    altitude, = struct.unpack_from('<i', p, idx)
                    idx += 4
                    output.append(str(altitude))  # Append altitude to output

                    # GPS status, satellites, horizontal accuracy, vertical accuracy are 8-bit integers
                    if revision > 0:  # These fields are valid only for certain revisions
                        gps_status, = struct.unpack_from('<b', p, idx)
                        idx += 1
                        output.append(str(gps_status))  # Append GPS status to output

                        satellites, = struct.unpack_from('<b', p, idx)
                        idx += 1
                        output.append(str(satellites))  # Append satellites count to output

                        horz_accuracy, = struct.unpack_from('<b', p, idx)
                        idx += 1
                        output.append(str(horz_accuracy))  # Append horizontal accuracy to output

                        vert_accuracy, = struct.unpack_from('<b', p, idx)
                        idx += 1
                        output.append(str(vert_accuracy))  # Append vertical accuracy to output

                else:
                    output.extend(['9'] * 9)       
            #print(delimiter.join(output))
            #print(output)
            file.write(delimiter.join(output) + "\n")  # Write packet to the file
            mgl_records.append(output)
            packets += 1
            if packets % 1000 == 0:
                print("p", end='')
            
        return mgl_records

#Execution Starts Here

delimiter_type = 'comma'
filename =  '2018-6-27-Enigma' #'OAR-Enigma'
fileext = 'rec'


parser = EnigmaLogParser(f"{filename}.{fileext}")
print("Scanning for first and last record, please wait")


    
oldest_pos, newest_pos, sections_seen, reversals = parser.find_first_last()

if oldest_pos == -1 or newest_pos == -1:
    print("Unable to locate starting and/or ending record\n")
else:
    print("Optional record sections present:")
    if sections_seen & RDAC1_PRESENT:
        print(" RDAC1")
    if sections_seen & RDAC2_PRESENT:
        print(" RDAC2")
    if sections_seen & ATTITUDE_PRESENT:
        print(" Attitude")
    if sections_seen & GPS_PRESENT:
        print(" GPS")

    # Write the output to a file called "filename.txt"
    with open(f"{filename}.txt", "w") as file:
        mgl_header = parser.print_header(delimiter_type, sections_seen)

        parser.bad_packets = 0

        if reversals > 1:
            print(
        f"\nWARNING: the record timestamps are inconsistent, there are {reversals} groups of\n"
         "contiguous records. All records will be dumped in the order they appear in\n"
        "the file instead of from oldest to newest.\n\n".format(reversals + 1)
            )
    
            parser.logfile.seek(0, 0)
        else:
            parser.logfile.seek(oldest_pos, 0)

        print(f"Dumping packets from:{parser.logfile.tell()} please wait")

        mgl_records1 = parser.process_packets(delimiter_type, sys.maxsize, sections_seen)
        mgl_records2 = 0

        if reversals <= 1 and newest_pos < oldest_pos:
            print(f"More packets from:  {parser.logfile.tell()} please wait")
            mgl_records2 = parser.process_packets(delimiter_type, oldest_pos, sections_seen)

# Creating a Pandas DataFrame using the headers and mgl_records
mgl_df = pd.DataFrame(mgl_records1, columns=mgl_header)

# Display the DataFrame
print(mgl_df)
parser.logfile.close()


Scanning for first and last record, please wait
ffffffffffffffffffffffffffffffffffffffffffffffbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbrp_EOF_i1rp_nonefl_breakOptional record sections present:
 RDAC1
 Attitude
 GPS
Dumping packets from:0 please wait
ppppppppppppppppppppppppppppppppppppppppppppppbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbrp_EOF_i1rp_none                        Time   alt  baro ASI TAS   VSI    g-s rotor   m-v  \
0      "2018/03/23 08:48:22"  1695  3123  32  32     4   99.9     0  13.1   
1      "2018/03/23 08:48:23"  1697  3123  39  40   309   11.3     0  13.1   
2      "2018/03/23 08:48:24"  1696  3123  44  45  -111  -35.6     0  13.1   
3      "2018/03/23 08:48:25"  1696  3123  49  50    53   83.0     0  13.2   
4      "2018/03/23 08:48:26"  1695  3123  53  54   -39  -99.9     0  13.2   
...                      ...   ...   ...  ..  ..   ...    ...   ...   ...   
46295  "2018/06/05 14:59:30"  1427  3157   0   0   -90    0.0     0  12.2   
46296  "2018/06/05 14:5

In [55]:
import folium
# Create a Folium map centered on Australia (latitude: -25.2744, longitude: 133.7751)
map_center = [-35.30131, 149.18610]
m = folium.Map(location=map_center, zoom_start=5)

# Loop through the DataFrame and add points to the map
for i, row in mgl_df.iloc[::360].iterrows():
    folium.Marker(
        location=[row['lat'], row['long']],
        popup=f"{row['Time']}",  # Customize popup as needed
        icon=folium.Icon(icon='info-sign', color='blue')  # Set icon and color
    ).add_to(m)

# Display the map (if you're in a Jupyter environment, this will render the map)
m.save('australia_map.html')  # Optionally, save the map to an HTML file

# To display in Jupyter Lab, simply return the map object
m
print(mgl_df)

                        Time   alt  baro ASI TAS   VSI    g-s rotor   m-v  \
0      "2018/03/23 08:48:22"  1695  3123  32  32     4   99.9     0  13.1   
1      "2018/03/23 08:48:23"  1697  3123  39  40   309   11.3     0  13.1   
2      "2018/03/23 08:48:24"  1696  3123  44  45  -111  -35.6     0  13.1   
3      "2018/03/23 08:48:25"  1696  3123  49  50    53   83.0     0  13.2   
4      "2018/03/23 08:48:26"  1695  3123  53  54   -39  -99.9     0  13.2   
...                      ...   ...   ...  ..  ..   ...    ...   ...   ...   
46295  "2018/06/05 14:59:30"  1427  3157   0   0   -90    0.0     0  12.2   
46296  "2018/06/05 14:59:31"  1429  3157   0   0    82    0.0     0  12.1   
46297  "2018/06/05 14:59:32"  1428  3157   0   0  -101    0.0     0  12.1   
46298  "2018/06/05 14:59:33"  1431  3157   0   0   170    0.0     0  12.2   
46299  "2018/06/05 14:59:34"  1430  3157   0   0   -69    0.0     0  12.2   

        b-v  ...    turn        lat       long   hdg  gs g-alt status sats 

In [51]:
map_center = [-35.30131, 149.18610]  # Centering the map on Australia
m = folium.Map(location=map_center, zoom_start=5)

# Add your point
folium.Marker(
    location=[-35.30131, 149.18610],  # Canberra coordinates for example
    popup="Canberra",
    icon=folium.Icon(icon='info-sign', color='blue')
).add_to(m)

m.save('australia_map.html')  # Save the map to view in browser
m

In [17]:
# Open the file in binary mode (or text mode if working with text files)
with open('2018-6-27-Enigma.rec', 'rb') as f:
    # Move the file pointer to the end of the file
    f.seek(0, 2)  # 0 bytes relative to the end of the file
    file_size = f.tell()  # Get the current position, which is the file size in bytes
    print(f"File size: {file_size} bytes")
    
    # Read the last byte
    f.seek(-1, 2)  # Move to 1 byte before the end of the file
    last_byte = f.read(1)  # Read the last byte
    print(f"Last byte: {last_byte}")


File size: 52428800 bytes
Last byte: b'\x00'


In [5]:
with open('2018-6-27-Enigma.rec', 'rb') as f:  # Open the file in binary mode
    packet = f.read(1024)  # Read the first 8 bytes

# Convert each byte to its hexadecimal representation and join them with a space
hex_output = ' '.join(f'{byte:02X}' for byte in packet)
# Set the block size (number of bytes per line)
block_size = 141

# Split the hex output into blocks of 'block_size' and print each block on a new line
hex_blocks = hex_output.split()  # Convert hex string to a list of hex values
for i in range(0, len(hex_blocks), block_size):
    # Print each block (8 hex values per line)
    print(' '.join(hex_blocks[i:i + block_size]))

payload = packet
# packet[0] is the total payload length + fudge factor
# packet[1] is an offset
sections_seen = 0
rdac1_len = 0
rdac2_len = 0
attitude_len = 0
gps_len = 0

revision = 2
p_offset = section_sizes[0][revision]+ HEADER_LEN + fudge_factors[revision]
p_idx = p_offset
payload_len = 141

# Parse sections
print(p_idx, payload[p_idx])
if p_idx < payload_len and payload[p_idx] == rdac1_data_tag:
    sections_seen |= RDAC1_PRESENT
    print('RADC 1')
    tag = payload[p_idx]
    p_idx += 1
    rdac1_len = payload[p_idx]
    p_idx += 1
    p_idx += rdac1_len

if p_idx < payload_len and payload[p_idx] == rdac2_data_tag:
    sections_seen |= RDAC2_PRESENT
    print('RADC 2')
    tag = payload[p_idx]
    p_idx += 1
    rdac2_len = payload[p_idx]
    p_idx += 1
    p_idx += rdac2_len

if p_idx < payload_len and payload[p_idx] == attitude_data_tag:
    sections_seen |= ATTITUDE_PRESENT
    print('ATT')
    tag = payload[p_idx]
    p_idx += 1
    attitude_len = payload[p_idx]
    p_idx += 1
    p_idx += attitude_len

if p_idx < payload_len and payload[p_idx] == gps_data_tag:
    sections_seen |= GPS_PRESENT
    print('GPS')
    tag = payload[p_idx]
    p_idx += 1
    gps_len = payload[p_idx]
    p_idx += 1
    p_idx += gps_len

if (p_idx == payload_len and
    ((sections_seen & RDAC1_PRESENT == 0) or rdac1_len == section_sizes[rdac1_section][revision]) and
    ((sections_seen & RDAC2_PRESENT == 0) or rdac2_len == section_sizes[rdac2_section][revision]) and
    ((sections_seen & ATTITUDE_PRESENT == 0) or attitude_len == section_sizes[attitude_section][revision]) and
    ((sections_seen & GPS_PRESENT == 0) or gps_len == section_sizes[gps_section][revision])
   ):
    print( payload_len, revision, sections_seen)

print( sections_seen, payload_len)

AA 55 8B 1E D6 7B 47 22 9F 06 00 00 33 0C 20 00 20 00 04 00 E7 03 00 80 83 7E 00 00 B4 00 0D 06 42 01 01 3F 9A 09 BC 08 C0 08 1A 00 3B 00 1E 12 AB 03 1E 00 1E 00 30 04 3B 00 18 00 00 00 03 19 17 02 AD 01 16 02 D2 01 19 02 0E 02 84 00 80 00 76 00 74 00 6F 00 73 00 AC 05 E1 01 E3 01 11 06 62 09 00 03 0E 0A 00 6F 00 F4 FF 18 05 82 00 0A 00 FC FF 04 18 8B 34 0D C2 A4 2F 15 43 27 05 00 00 21 00 00 00 5B 07 00 00 03 0A 05 07
AA 55 8B 1E D7 7B 47 22 A1 06 00 00 33 0C 27 00 28 00 35 01 71 00 00 80 83 7D 00 00 B4 00 0D 06 86 01 01 3F 96 0A C0 08 C0 08 1A 00 29 00 1E 12 AD 03 1E 00 1E 00 30 04 3A 00 25 00 00 00 16 19 1C 02 B6 01 1E 02 DD 01 28 02 17 02 86 00 82 00 76 00 74 00 6F 00 73 00 A7 05 AE 03 E4 01 87 08 63 09 00 03 0E 0A 00 76 00 F9 FF 36 05 85 00 0A 00 F2 FF 04 18 A4 34 0D C2 AD 2F 15 43 1C 05 00 00 24 00 00 00 5B 07 00 00 03 0B 05 07
AA 55 8B 1E D8 7B 47 22 A0 06 00 00 33 0C 2C 00 2D 00 91 FF 9C FE 00 80 83 7D 00 00 63 00 0D 06 B8 01 01 3F EF 0A BD 08 C0 08 1A 00 38 00 73 10 A3 03 1E 0

In [ ]:
### timestamp = 569203200 #512457686
#           0    1    2    3    4   5   6     7
packet = [0xaa,0x55,0x8b,0x1e,0xd6,0x7b,0x47,0x22,0x9f]
timestamp = (
                packet[7] << 24 |
                packet[6] << 16 |
                packet[5] << 8 |
                packet[4])
base_time = datetime.datetime(2000, 1, 1)
adj_time = base_time + datetime.timedelta(seconds=timestamp)
time_str = adj_time.strftime("%d/%m/%Y %H:%M:%S")
output = [f'"{time_str}"']
print(f'Stamp:{timestamp}   Base:{base_time}  Output:{output}')
print(packet[2])